# Machine Learning avec Spark

## Pourquoi Spark pour ML

Spark est un moteur d'analyse unifié qui fournit un écosystème pour l'ingestion de données, le feature ingineering, la modelisation et le déploiement. Sans Spark, les développeurs auraient besoin de nombreux outils disparates pour accomplir cet ensemble de tâches, et pourraient encore se débattre avec l'évolutivité.

Avec `spark.ml`, les data scientist peuvent utiliser un seul écosystème pour la préparation des données et la construction de modèles, sans avoir besoin de sous-échantillonner leurs données pour les faire tenir sur une seule machine. `spark.ml` se concentre sur la mise à l'échelle `O(n)`, où le modèle s'échelonne linéairement avec le nombre de points de données dont vous disposez, de sorte qu'il peut s'adapter à des quantités massives de données. Si vous avez déjà utilisé scikit-learn, de nombreuses API de spark.ml vous sembleront assez familières, mais il existe quelques différences subtiles dont nous parlerons.

## Spark MLlib

Spark MLlib est le module machine learning de l'ecosysteme Spark. Il est caractersise par les concepte suivants:

### Transformer

Il accepte un DataFrame en entrée, et renvoie un nouveau DataFrame avec une ou plusieurs colonnes en annexe. Les transformateurs n'apprennent aucun paramètre à partir de vos données et appliquent simplement des transformations basées sur des règles pour préparer les données à l'entraînement du modèle ou pour générer des prédictions à l'aide d'un modèle MLlib entraîné. Ils disposent d'une méthode `.transform()`.

### Estimator

Il apprend (ou "ajuste") les paramètres de votre DataFrame via une méthode `.fit()` et renvoie un modèle, qui est un transformer.

### Pipeline

Il organise une série de transformateurs et d'estimateurs en un seul modèle. Alors que les pipelines eux-mêmes sont des estimateurs, la sortie de `pipeline.fit()` renvoie un `PipelineModel`, un transformateur.

Nous allons faire une etude de cas pratique en suivant le machine learning worklof.

## Business problem

Notre objectif est de construire un modèle permettant de prédire les prix des locations de logement Airbnb à la nuitée pour les annonces dans la ville de San Francisco.

## Machine learning problem

Il s'agit d'un problème de régression, car le prix est une variable continue. 

## Source de donnees

Notre avons juger pertinent d'utiliser l'ensemble des données sur le logement à San Francisco provenant d'Inside Airbnb. Il contient des informations sur les locations d'Airbnb à San Francisco, telles que le nombre de chambres à coucher, l'emplacement, les review score, etc. Ces donnees sont telechargeable sur le site d'[Inside Airbnb](http://insideairbnb.com/get-the-data.html). Cependant nous les avons deja telecharge et placer dans le repertoire data du cours.

## Ingestion des donnees

In [ ]:
import $ivy.`org.apache.spark::spark-sql:2.4.5` // Or use any other 2.x version here
import $ivy.`org.apache.spark::spark-mllib:2.4.5`
import org.apache.log4j.{Level, Logger}
import org.apache.spark.sql._

Logger.getLogger("org").setLevel(Level.OFF)

val spark = {
  NotebookSparkSession.builder()
    .master("local[4]")
    .getOrCreate()
}

In [ ]:
val filePath = "data/sf_airbnb/raw/sf-airbnb.csv"

val rawDF = spark.read
  .option("header", "true")
  .option("multiLine", "true")
  .option("inferSchema", "true")
  .option("escape", "\"")
  .csv(filePath)

In [ ]:
rawDF.printSchema

In [ ]:
rawDF.columns.length

## Selection empirique de features

Nous allons conserver que les colonnes qui nous semblent pertinentes pour notre analyse:

In [5]:
val baseDF = rawDF.select(
  "host_is_superhost",
  "cancellation_policy",
  "instant_bookable",
  "host_total_listings_count",
  "neighbourhood_cleansed",
  "latitude",
  "longitude",
  "property_type",
  "room_type",
  "accommodates",
  "bathrooms",
  "bedrooms",
  "beds",
  "bed_type",
  "minimum_nights",
  "number_of_reviews",
  "review_scores_rating",
  "review_scores_accuracy",
  "review_scores_cleanliness",
  "review_scores_checkin",
  "review_scores_communication",
  "review_scores_location",
  "review_scores_value",
  "price")

baseDF.cache().count

count at cmd4.sc:27

1 / 1

count at cmd4.sc:27

1 / 1

baseDF: DataFrame = [host_is_superhost: string, cancellation_policy: string ... 22 more fields]
res4_1: Long = 7151L

In [ ]:
baseDF.show(5)

Par des soucis de rendu, nous allons juste visualiser une partie du dataframe baseDF:

In [7]:
baseDF.select(
    "host_is_superhost",
    "latitude",
    "longitude",
    "accommodates",
    "bedrooms",
    "review_scores_rating",
    "price"
).show

show at cmd6.sc:8

1 / 1

+-----------------+--------+----------+------------+--------+--------------------+-------+
|host_is_superhost|latitude| longitude|accommodates|bedrooms|review_scores_rating|  price|
+-----------------+--------+----------+------------+--------+--------------------+-------+
|                t|37.76931|-122.43386|           3|       1|                  97|$170.00|
|                f|37.74511|-122.42102|           5|       2|                  98|$235.00|
|                f|37.76669| -122.4525|           2|       1|                  85| $65.00|
|                f|37.76487|-122.45183|           2|       1|                  93| $65.00|
|                f|37.77525|-122.43637|           5|       2|                  97|$785.00|
|                f|37.78471|-122.44555|           6|       2|                  90|$255.00|
|                t|37.75919|-122.42237|           3|       1|                  98|$139.00|
|                f|37.76259|-122.40543|           2|       1|                  94|$135.00|

## Preprocessing des donnees

### Corriger le typage des donnees

Regardez le schéma ci-dessus. Vous remarquerez que le champ prix a été repris sous forme de chaîne. Pour notre tâche, il doit s'agir d'un champ numérique (double type). 

Corrigeons cela:

In [ ]:
import org.apache.spark.sql.functions.{col, translate}

val fixedPriceDF = baseDF.withColumn("price", translate(col("price"), "$,", "").cast("double"))

fixedPriceDF.select("price").show(5)

In [8]:
fixedPriceDF.select("price").printSchema

root
 |-- price: double (nullable = true)



### Statistiques sommaires

In [ ]:
fixedPriceDF.printSchema

La methode `.summary()` nous donnes quelques statistiques descriptives sur notre dataset:

In [11]:
fixedPriceDF
  .select(
      "host_total_listings_count",
      "latitude",
      "longitude"
  )
  .summary()
  .show

summary at cmd10.sc:5

1 / 1

summary at cmd10.sc:5

1 / 1

+-------+-------------------------+--------------------+--------------------+
|summary|host_total_listings_count|            latitude|           longitude|
+-------+-------------------------+--------------------+--------------------+
|  count|                     7151|                7151|                7151|
|   mean|        52.56957068941407|   37.76580945042649| -122.43052552230478|
| stddev|       177.37165167124357|0.022527191846014046|0.026791775802673057|
|    min|                        0|            37.70743|          -122.51306|
|    25%|                        1|            37.75111|          -122.44295|
|    50%|                        2|            37.76755|          -122.42547|
|    75%|                     1199|            37.81031|          -122.36979|
|    max|                     1199|            37.81031|          -122.36979|
+-------+-------------------------+--------------------+--------------------+



In [12]:
fixedPriceDF
  .select(
      "accommodates",
      "bathrooms",
      "bedrooms",
      "beds"
  )
  .summary()
  .show

summary at cmd11.sc:6

1 / 1

summary at cmd11.sc:6

1 / 1

+-------+------------------+------------------+------------------+------------------+
|summary|      accommodates|         bathrooms|          bedrooms|              beds|
+-------+------------------+------------------+------------------+------------------+
|  count|              7151|              7130|              7149|              7144|
|   mean|3.2009509159558105| 1.328962131837307|1.3425653937613653|1.7648376259798433|
| stddev|1.9146923115947185|0.7945555125892707|0.9326852094201303| 1.176852628831775|
|    min|                 1|               0.0|                 0|                 0|
|    25%|                 2|               1.0|                 1|                 1|
|    50%|                 2|               1.0|                 1|                 1|
|    75%|                16|              14.0|                14|                14|
|    max|                16|              14.0|                14|                14|
+-------+------------------+------------------+-------

In [13]:
fixedPriceDF
  .select(
      "minimum_nights",
      "number_of_reviews",
      "review_scores_rating",
      "review_scores_accuracy"
  )
  .summary()
  .show

summary at cmd12.sc:6

1 / 1

summary at cmd12.sc:6

1 / 1

+-------+------------------+-----------------+--------------------+----------------------+
|summary|    minimum_nights|number_of_reviews|review_scores_rating|review_scores_accuracy|
+-------+------------------+-----------------+--------------------+----------------------+
|  count|              7151|             7151|                5730|                  5726|
|   mean|14000.302335337716|43.52915676129213|   95.54694589877836|     9.775585050646175|
| stddev|1182541.9078980184|72.51922886627213|    6.93515172677721|    0.6651167005432524|
|    min|                 1|                0|                  20|                     2|
|    25%|                 2|                1|                  94|                    10|
|    50%|                 4|               11|                  98|                    10|
|    75%|         100000000|              677|                 100|                    10|
|    max|         100000000|              677|                 100|                    10|

In [14]:
fixedPriceDF
  .select(
      "review_scores_cleanliness",
      "review_scores_checkin",
      "review_scores_communication"
  )
  .summary()
  .show

summary at cmd13.sc:5

1 / 1

summary at cmd13.sc:5

1 / 1

+-------+-------------------------+---------------------+---------------------------+
|summary|review_scores_cleanliness|review_scores_checkin|review_scores_communication|
+-------+-------------------------+---------------------+---------------------------+
|  count|                     5727|                 5724|                       5728|
|   mean|        9.624934520691461|    9.870020964360586|          9.841305865921788|
| stddev|       0.7683441047916056|   0.4979945098078443|         0.5794253386655072|
|    min|                        2|                    2|                          2|
|    25%|                        9|                   10|                         10|
|    50%|                       10|                   10|                         10|
|    75%|                       10|                   10|                         10|
|    max|                       10|                   10|                         10|
+-------+-------------------------+-------------------

In [15]:
fixedPriceDF
  .select(
      "review_scores_location",
      "review_scores_value",
      "price"
  )
  .summary()
  .show

summary at cmd14.sc:5

1 / 1

summary at cmd14.sc:5

1 / 1

+-------+----------------------+-------------------+------------------+
|summary|review_scores_location|review_scores_value|             price|
+-------+----------------------+-------------------+------------------+
|  count|                  5724|               5723|              7151|
|   mean|      9.64937106918239|   9.40573125982876| 213.6540344007831|
| stddev|    0.7198160859533849| 0.7969151479310892|313.28222046853125|
|    min|                     2|                  2|               0.0|
|    25%|                     9|                  9|             100.0|
|    50%|                    10|                 10|             150.0|
|    75%|                    10|                 10|           10000.0|
|    max|                    10|                 10|           10000.0|
+-------+----------------------+-------------------+------------------+



### Gestion des valeurs manquantes

#### Identification des colonnes manquantes

In [10]:
fixedPriceDF.columns.map(c => (c, fixedPriceDF.filter(col(c).isNull || col(c).isNaN).count()))
    .filter(_._2 > 0)
//     .map(_._1)

count at cmd9.sc:1

1 / 1

count at cmd9.sc:1

1 / 1

count at cmd9.sc:1

...

count at cmd9.sc:1

1 / 1

count at cmd9.sc:1

1 / 1

res9: Array[(String, Long)] = Array(
  ("bathrooms", 21L),
  ("bedrooms", 2L),
  ("beds", 7L),
  ("review_scores_rating", 1421L),
  ("review_scores_accuracy", 1425L),
  ("review_scores_cleanliness", 1424L),
  ("review_scores_checkin", 1427L),
  ("review_scores_communication", 1423L),
  ("review_scores_location", 1427L),
  ("review_scores_value", 1428L)
)

#### Imputation

SparkML's Imputer exige que tous les champs soient de type double. Nous allons donc faire en sorte que tous les champs entiers soient de type double:

In [11]:
baseDF.schema

res10: types.StructType = StructType(
  StructField("host_is_superhost", StringType, true, {}),
  StructField("cancellation_policy", StringType, true, {}),
  StructField("instant_bookable", StringType, true, {}),
  StructField("host_total_listings_count", IntegerType, true, {}),
  StructField("neighbourhood_cleansed", StringType, true, {}),
  StructField("latitude", DoubleType, true, {}),
  StructField("longitude", DoubleType, true, {}),
  StructField("property_type", StringType, true, {}),
  StructField("room_type", StringType, true, {}),
  StructField("accommodates", IntegerType, true, {}),
  StructField("bathrooms", DoubleType, true, {}),
  StructField("bedrooms", IntegerType, true, {}),
  StructField("beds", IntegerType, true, {}),
  StructField("bed_type", StringType, true, {}),
  StructField("minimum_nights", IntegerType, true, {}),
  StructField("number_of_reviews", IntegerType, true, {}),
  StructField("review_scores_rating", IntegerType, true, {}),
  StructField("review_scores

In [13]:
import org.apache.spark.sql.functions.col
import org.apache.spark.sql.types.IntegerType

val integerColumns = for (x <- baseDF.schema.fields if (x.dataType == IntegerType)) yield x.name  
var doublesDF = fixedPriceDF

for (c <- integerColumns)
  doublesDF = doublesDF.withColumn(c, col(c).cast("double"))

val columns = integerColumns.mkString("\n - ")
println(s"Columns converted from Integer to Double:\n - $columns \n")
println("*-"*80)

Columns converted from Integer to Double:
 - host_total_listings_count
 - accommodates
 - bedrooms
 - beds
 - minimum_nights
 - number_of_reviews
 - review_scores_rating
 - review_scores_accuracy
 - review_scores_cleanliness
 - review_scores_checkin
 - review_scores_communication
 - review_scores_location
 - review_scores_value 

*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-


import org.apache.spark.sql.functions.col

import org.apache.spark.sql.types.IntegerType


integerColumns: Array[String] = Array(
  "host_total_listings_count",
  "accommodates",
  "bedrooms",
  "beds",
  "minimum_nights",
  "number_of_reviews",
  "review_scores_rating",
  "review_scores_accuracy",
  "review_scores_cleanliness",
  "review_scores_checkin",
  "review_scores_communication",
  "review_scores_location",
  "review_scores_value"
)
doublesDF: DataFrame = [host_is_superhost: string, cancellation_policy: string ... 22 more fields]
columns: String = """host_total_listings_count
 - accommodates
 - bedrooms
 - beds
 - minimum_nights
 - number_of_reviews
 - review_scores_rating
 - review_scores_accuracy
 - review_scores_cleanliness
 - review_scores_checkin
 - review_scores_communication
 - review_scores_location
 - review_scores_value"""

Ajoutez une variable fictive si nous voulons imputer une valeur.

In [19]:
val missingCols = fixedPriceDF.columns.map(c => (c, fixedPriceDF.filter(col(c).isNull || col(c).isNaN).count()))
    .filter(_._2 > 0)
    .map(_._1)

count at cmd18.sc:1

1 / 1

count at cmd18.sc:1

1 / 1

count at cmd18.sc:1

...

count at cmd18.sc:1

1 / 1

count at cmd18.sc:1

1 / 1

missingCols: Array[String] = Array(
  "bathrooms",
  "bedrooms",
  "beds",
  "review_scores_rating",
  "review_scores_accuracy",
  "review_scores_cleanliness",
  "review_scores_checkin",
  "review_scores_communication",
  "review_scores_location",
  "review_scores_value"
)

In [20]:
missingCols

res19: Array[String] = Array(
  "bathrooms",
  "bedrooms",
  "beds",
  "review_scores_rating",
  "review_scores_accuracy",
  "review_scores_cleanliness",
  "review_scores_checkin",
  "review_scores_communication",
  "review_scores_location",
  "review_scores_value"
)

In [21]:
import org.apache.spark.sql.functions.when

for (c <- missingCols)
  doublesDF = doublesDF.withColumn(c + "_na", when(col(c).isNull, 1.0).otherwise(0.0))

import org.apache.spark.sql.functions.when


imputeCols: Array[String] = Array(
  "bathrooms",
  "bedrooms",
  "beds",
  "review_scores_rating",
  "review_scores_accuracy",
  "review_scores_cleanliness",
  "review_scores_checkin",
  "review_scores_communication",
  "review_scores_location",
  "review_scores_value"
)

In [22]:
doublesDF.printSchema

root
 |-- host_is_superhost: string (nullable = true)
 |-- cancellation_policy: string (nullable = true)
 |-- instant_bookable: string (nullable = true)
 |-- host_total_listings_count: double (nullable = true)
 |-- neighbourhood_cleansed: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- property_type: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- accommodates: double (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- bedrooms: double (nullable = true)
 |-- beds: double (nullable = true)
 |-- bed_type: string (nullable = true)
 |-- minimum_nights: double (nullable = true)
 |-- number_of_reviews: double (nullable = true)
 |-- review_scores_rating: double (nullable = true)
 |-- review_scores_accuracy: double (nullable = true)
 |-- review_scores_cleanliness: double (nullable = true)
 |-- review_scores_checkin: double (nullable = true)
 |-- review_scores_communication: double (nullable = true

Imputons par la medianne:

In [23]:
import org.apache.spark.ml.feature.Imputer

val imputer = new Imputer()
  .setStrategy("median")
  .setInputCols(imputeCols)
  .setOutputCols(imputeCols)

val imputedDF = imputer.fit(doublesDF).transform(doublesDF)

approxQuantile at Imputer.scala:151

1 / 1

head at Imputer.scala:216

1 / 1

head at Imputer.scala:216

3 / 3

import org.apache.spark.ml.feature.Imputer


imputer: Imputer = imputer_3fa4b386d075
imputedDF: DataFrame = [host_is_superhost: string, cancellation_policy: string ... 32 more fields]

Verifions qu'il n'y a plus de valeurs manquantes apres imputation:

In [24]:
imputedDF.columns.map(c => (c, imputedDF.filter(col(c).isNull || col(c).isNaN).count()))
    .filter(_._2 > 0)

count at cmd23.sc:1

1 / 1

count at cmd23.sc:1

1 / 1

count at cmd23.sc:1

...

count at cmd23.sc:1

1 / 1

count at cmd23.sc:1

1 / 1

res23: Array[(String, Long)] = Array()

### Gestion des outliers

Examinons les valeurs min et max de la colonne price :

In [25]:
imputedDF.select("price").describe().show

describe at cmd24.sc:1

1 / 1

describe at cmd24.sc:1

1 / 1

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|              7151|
|   mean| 213.6540344007831|
| stddev|313.28222046853125|
|    min|               0.0|
|    max|           10000.0|
+-------+------------------+



Il existe des logements super chèrs. Mais nous avons décider de les garder. Ce pendant, nous allons filtrer les Airbnbs "gratuits".

Voyons d'abord combien d'annonces nous pouvons trouver là où le prix est nul:

In [26]:
imputedDF.filter(col("price") === 0).count

count at cmd25.sc:1

1 / 1

count at cmd25.sc:1

1 / 1

res25: Long = 1L

Gardons maintenant que les rangées ayant un prix strictement positif:

In [27]:
val posPricesDF = imputedDF.filter(col("price") > 0)

posPricesDF: Dataset[Row] = [host_is_superhost: string, cancellation_policy: string ... 32 more fields]

Examinons maintenant les valeurs min et max de la colonne minimum_nights :

In [28]:
posPricesDF.select("minimum_nights").describe().show

describe at cmd27.sc:1

1 / 1

describe at cmd27.sc:1

1 / 1

+-------+------------------+
|summary|    minimum_nights|
+-------+------------------+
|  count|              7150|
|   mean| 14002.25986013986|
| stddev|1182624.6002248244|
|    min|               1.0|
|    max|             1.0E8|
+-------+------------------+



In [29]:
posPricesDF
  .groupBy("minimum_nights").count()
  .orderBy(col("count").desc, col("minimum_nights")
).show

show at cmd28.sc:3

1 / 1

show at cmd28.sc:3

200 / 200

+--------------+-----+
|minimum_nights|count|
+--------------+-----+
|          30.0| 2757|
|           2.0| 1455|
|           1.0| 1251|
|           3.0|  822|
|           4.0|  270|
|           5.0|  176|
|          31.0|  133|
|           7.0|   72|
|          60.0|   32|
|           6.0|   31|
|          32.0|   31|
|          90.0|   28|
|         180.0|   28|
|          45.0|    7|
|         365.0|    7|
|         120.0|    6|
|          14.0|    4|
|          10.0|    3|
|          40.0|    3|
|          28.0|    2|
+--------------+-----+
only showing top 20 rows



Un séjour maximum d'un an semble être une limite raisonnable ici. Filtrons les enregistrements où le nombre de nuits minimum est supérieur à 365 :

In [30]:
val cleanDF = posPricesDF.filter(col("minimum_nights") <= 365)

cleanDF: Dataset[Row] = [host_is_superhost: string, cancellation_policy: string ... 32 more fields]

In [31]:
cleanDF.select(
    "instant_bookable",
    "latitude",
    "longitude",
    "bedrooms_na",
    "beds_na",
    "price"
).show

show at cmd30.sc:7

1 / 1

+----------------+--------+----------+-----------+-------+-----+
|instant_bookable|latitude| longitude|bedrooms_na|beds_na|price|
+----------------+--------+----------+-----------+-------+-----+
|               t|37.76931|-122.43386|        0.0|    0.0|170.0|
|               f|37.74511|-122.42102|        0.0|    0.0|235.0|
|               f|37.76669| -122.4525|        0.0|    0.0| 65.0|
|               f|37.76487|-122.45183|        0.0|    0.0| 65.0|
|               f|37.77525|-122.43637|        0.0|    0.0|785.0|
|               f|37.78471|-122.44555|        0.0|    0.0|255.0|
|               t|37.75919|-122.42237|        0.0|    0.0|139.0|
|               f|37.76259|-122.40543|        0.0|    0.0|135.0|
|               f|37.75874|-122.41327|        0.0|    0.0|265.0|
|               f|37.77187|-122.43859|        0.0|    0.0|177.0|
|               f|37.77355|-122.42436|        0.0|    0.0|194.0|
|               f|37.78574|-122.40798|        0.0|    0.0|139.0|
|               f|37.7701

In [32]:
cleanDF.printSchema

root
 |-- host_is_superhost: string (nullable = true)
 |-- cancellation_policy: string (nullable = true)
 |-- instant_bookable: string (nullable = true)
 |-- host_total_listings_count: double (nullable = true)
 |-- neighbourhood_cleansed: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- property_type: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- accommodates: double (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- bedrooms: double (nullable = true)
 |-- beds: double (nullable = true)
 |-- bed_type: string (nullable = true)
 |-- minimum_nights: double (nullable = true)
 |-- number_of_reviews: double (nullable = true)
 |-- review_scores_rating: double (nullable = true)
 |-- review_scores_accuracy: double (nullable = true)
 |-- review_scores_cleanliness: double (nullable = true)
 |-- review_scores_checkin: double (nullable = true)
 |-- review_scores_communication: double (nullable = true

OK, nos données sont nettoyées maintenant. Enregistrons ce DataFrame dans un fichier afin de pouvoir commencer à construire des modèles avec lui:

In [33]:
val outputPath = "data/sf_airbnb/processed/sf-airbnb-clean.parquet"

cleanDF.write.mode("overwrite").parquet(outputPath)

parquet at cmd32.sc:3

1 / 1

outputPath: String = "data/sf_airbnb/processed/sf-airbnb-clean.parquet"

## prediction du prix de loyer

In [ ]:
airnbData = baseDf

In [ ]:
#importation du transformer
from pyspark.ml.feature import VectorAssembler

In [ ]:
vecAssemb = VectorAssembler(inputCols=["bedrooms"], outputCol="features")

In [ ]:
vecTrans = vecAssemb.transform(airnbData)

In [ ]:
vectTrans.select("bedrooms", "features", "price").show(10)


In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
#instanciation
lr = LinearRegression(featuresCol="features", labelCol="price")

In [ ]:
lrModel = lr.fit(vecTrainDF)

In [ ]:
#application de la formule y = mx + b 
m = round(lrModel.coefficients[0], 2)
b = round(lrModel.intercept, 2)

In [ ]:
from pyspark.ml import Pipeline 

In [ ]:
pipeline = Pipeline(stages=[vecAssembler, lr])
pipelineModel = pipeline.fit(airnbData)

In [ ]:
test = baseDf
predictDf = pipelineModel.transform(test)
predictDf.select("bedrooms", "features", "price", "prediction").show(10)